<a href="https://colab.research.google.com/github/Davjot/PlantDoc/blob/main/image_plantdoc_dn201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
!wget --no-check-certificate \
  https://storage.googleapis.com/tensorflow/keras-applications/densenet/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-04-06 14:14:30--  https://storage.googleapis.com/tensorflow/keras-applications/densenet/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74836368 (71M) [application/octet-stream]
Saving to: ‘densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5’

densenet201_weights 100%[===================>]  71.37M   116MB/s    in 0.6s    

2023-04-06 14:14:30 (116 MB/s) - ‘densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [74836368/74836368]



In [3]:
from tensorflow.keras.applications import DenseNet201

local_weights_file = '/content/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = DenseNet201(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet')
pre_trained_model.load_weights(local_weights_file)

74836368/74836368 [==============================] - 1s 0us/step


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
base_dir = '/content/drive/MyDrive/Plantdoc'
train_dir = os.path.join(base_dir)
validation_dir = os.path.join(base_dir)

In [6]:
train_apple_rust_leaf_dir = os.path.join(train_dir, 'Apple rust leaf')
validation_apple_rust_leaf_dir = os.path.join(validation_dir, 'Apple rust leaf')
train_blueberry_leaf_dir = os.path.join(train_dir, 'Blueberry leaf')
validation_blueberry_leaf_dir = os.path.join(validation_dir, 'Blueberry leaf')
#train_apple_rust_leaf_fnames = os.listdir(train_apple_rust_leaf_dir) 
#training set
train_apple_leaf_dir = os.path.join(train_dir, 'Apple leaf')
train_apple_scab_leaf_dir = os.path.join(train_dir, 'Apple scab leaf')
train_bell_pepper_leaf_dir = os.path.join(train_dir, 'Bell pepper leaf')
train_bell_pepper_leaf_spot_dir = os.path.join(train_dir, 'Bell pepper leaf spot')
train_cherry_leaf_dir = os.path.join(train_dir, 'Cherry leaf')
train_corn_gray_leaf_spot_dir = os.path.join(train_dir, 'Corn Gray leaf spot')

#validation set
validation_apple_leaf_dir = os.path.join(validation_dir, 'Apple leaf')
validation_apple_scab_leaf_dir = os.path.join(validation_dir, 'Apple scab leaf')
validation_bell_pepper_leaf_dir = os.path.join(validation_dir, 'Bell pepper leaf')
validation_bell_pepper_leaf_spot_dir = os.path.join(validation_dir, 'Bell pepper leaf spot')
validation_cherry_leaf_dir = os.path.join(validation_dir, 'Cherry leaf')
validation_corn_gray_leaf_spot_dir = os.path.join(validation_dir, 'Corn Gray leaf spot')

In [7]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=8,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

Found 80 images belonging to 8 classes.
Found 80 images belonging to 8 classes.


In [10]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [11]:
# create the base pre-trained model
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape = (224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(8, activation='softmax')(x)

In [12]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [13]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['acc'])

In [14]:
import tensorflow as tf
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5', save_weights_only = True),
    tf.keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/logdir_plantdoc_dn201'),
]

In [15]:
history = model.fit(
      train_generator,
      epochs=5,
      validation_data=validation_generator,
      verbose=2,
      callbacks = [my_callbacks])

Epoch 1/5
10/10 - 99s - loss: 2.2262 - acc: 0.2500 - val_loss: 1.4852 - val_acc: 0.5500 - 99s/epoch - 10s/step
Epoch 2/5
10/10 - 65s - loss: 1.6518 - acc: 0.4125 - val_loss: 1.0891 - val_acc: 0.6375 - 65s/epoch - 6s/step
Epoch 3/5
10/10 - 64s - loss: 1.1151 - acc: 0.6000 - val_loss: 0.6982 - val_acc: 0.8500 - 64s/epoch - 6s/step
Epoch 4/5
10/10 - 65s - loss: 0.8314 - acc: 0.7000 - val_loss: 0.6172 - val_acc: 0.8375 - 65s/epoch - 6s/step
Epoch 5/5
10/10 - 67s - loss: 0.7788 - acc: 0.7375 - val_loss: 0.3993 - val_acc: 0.9000 - 67s/epoch - 7s/step
